In [ ]:
%pip install scikit-learn


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from qdrant_client import QdrantClient, models
import numpy as np
from rich import print

## Collection Setup

In [22]:
import ollama

# Initialize Ollama client
oclient = ollama.Client(host="localhost")
# Create a vector for the Component node
response = oclient.embeddings(model="nomic-embed-text", prompt="Hello, world")
embeddings = response["embedding"]
# 2. Convert to Qdrant sparse vectors
COLLECTION_NAME = "demo_collection"
client = QdrantClient(host="localhost", port="6333")

if client.collection_exists(COLLECTION_NAME):
    client.delete_collection(COLLECTION_NAME)

client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config={
        "text-dense": models.VectorParams(
            size=len(embeddings), distance=models.Distance.COSINE
        )
    },
    sparse_vectors_config={
        "text-sparse": models.SparseVectorParams(
            index=models.SparseIndexParams(on_disk=False)
        )
    },
)

True

## Data Loading

In [6]:
from typing import List, Optional
from pydantic_xml import BaseXmlModel, attr, element


class NamePresentation(BaseXmlModel):
    edt: Optional[str] = attr("edt")
    value: str = ""


class Description(BaseXmlModel):
    edt: Optional[str] = attr("edt")
    value: str = ""


class ProductVariantConditionRef(BaseXmlModel):
    ref: Optional[str] = attr("ref")
    value: str = ""


class Server(BaseXmlModel):
    canAddress: str = element("CanAddress")
    propertyName: str = element("PropertyName")


class Property(BaseXmlModel):
    server: Server = element("Server")


class PropertyGroup(BaseXmlModel):
    name: str = element("Name")
    namePresentation: NamePresentation = element("NamePresentation")
    description: Optional[Description] = element("Description", default=None)  # ✅ Added
    property: List[Property] = element("Property")


class FunctionPropertyGroup(BaseXmlModel, tag="FunctionPropertyGroup"):
    name: str = element("Name")
    namePresentation: NamePresentation = element("NamePresentation")
    description: Optional[Description] = element("Description", default=None)
    productVariantConditionRef: Optional[ProductVariantConditionRef] = element("ProductVariantConditionRef", default=None)
    ufNumber: List[int] = element("UFNumber")

    propertyGroup: Optional[List[PropertyGroup]] = element("PropertyGroup", default=None)
    property: Optional[List[Property]] = element("Property", default=None)

    class Config:
        xml_ns = {
            "xsi": "http://www.w3.org/2001/XMLSchema-instance"
        }

In [8]:
from transformers import AutoTokenizer

# Load the Qwen tokenizer
tokenizer = AutoTokenizer.from_pretrained("NovaSky-AI/Sky-T1-32B-Flash", trust_remote_code=True)

/home/rajesh/Method_Engineer_Support_Tool/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [10]:
# load all the physical quantities from the ./data/Function-Parameters/PhysicalQuantity/ folder
from pathlib import Path

physical_quantity_files = Path("./data/Function-Parameters/FunctionViewAdjust").glob(
    "*.xml"
)

# make sure their names start with PhysicalQuantity
physical_quantity_files = [
    file
    for file in physical_quantity_files
    if file.name.startswith("FunctionPropertyGroup")
]

physical_quantities = []
token_count = 0
print("total files: ", len(physical_quantity_files))
funciton_properties_details = ""

function_group_objects = []

for file in physical_quantity_files:
    with file.open("rb") as f:
        xml_data = f.read()
        obj = FunctionPropertyGroup.from_xml(xml_data)
        physical_quantities.append(obj)

        model_json = obj.model_dump_json(indent=2)

        input_ids = tokenizer(model_json)["input_ids"]
        token_count += len(input_ids)

        funciton_properties_details += "\n\nFUNCTION PROPERTY GROUP " + obj.name + "\n"
        if obj.property:
            for prop in obj.property:
                funciton_properties_details += prop.server.propertyName + "\n"
        if obj.propertyGroup:
            for prop_group in obj.propertyGroup:
                funciton_properties_details += prop_group.name + "\n"
                for prop in prop_group.property:
                    funciton_properties_details += prop.server.propertyName + "\n"

        function_group_objects.append(obj)

print(f"Token count: {token_count}")

# tokens for all physical quantities
tokens_count_function_properties = len(
    tokenizer(funciton_properties_details)["input_ids"]
)

print(f"Token count for all function properties: {tokens_count_function_properties}")
# print(physical_quantities)

total files:  164
Token count: 82831
Token count for all function properties: 15219


In [ ]:
# # 0. Sample data
# documents = [
#     "Reading Light",
#     "Rear Cam Disp",
#     "GPS Navigation",
#     "GPS Panel",
#     "Heated Seats",
#     "Read Lite",
# ]

# # 1. Fit TF-IDF vectorizer with char n-grams
# vectorizer = TfidfVectorizer(analyzer='char_wb', ngram_range=(2, 4))
# X_sparse = vectorizer.fit_transform(documents)

documents = []
for obj in function_group_objects:
    document = ""

    if obj.property:
        for prop in obj.property:
            document += prop.server.propertyName + "\n"
    if obj.propertyGroup:
        for prop_group in obj.propertyGroup:
            document += prop_group.name + "\n"
            for prop in prop_group.property:
                document += prop.server.propertyName + "\n"
    print(document)
    documents.append(document)

# # 1. Fit TF-IDF vectorizer with char n-grams
vectorizer = TfidfVectorizer(analyzer='char_wb', ngram_range=(2, 4))
X_sparse = vectorizer.fit_transform(documents)


PVGLight
Approach_light

numberOfTanksBus
totalTankVolumeBus
numberOfTanksExtra
totalTankVolumeExtra

PTOGearDeac

Horn_Var
Horn_Source

OptionalOutputSignalsInC493
Out_1
Out_2
Out_3
Out_4
Out_5
Out_6
Out_1mode
Out_2mode
Out_3mode
Out_4mode
Out_5mode
Out_EngSpd
Out_Freq1
Out_Freq2
Out_Freq3
Out_Freq4
Out_Freq5
Out_Freq6
AdaptationToBodyworkGauge
OilPressure
EngineTemperature
FuelLevel
EngineLoad
CircuitPressure1
CircuitPressure2
TransOilTemp
RetOilTemp

Stop_Var
Stop_Source
SLEmergStop
EMERG_STOP_SPEED_COND
BCILimitEngineStop

PowerTakeOffSlot1
PTO1_Var
PTO1_Switch
PTO1_Feedback
PTO1_Lamp
CAN15_BB_LampConfig
PTO1_Act
PTO1_Neut
PTO1_N_Time
PTO1_Park
PTO1_nOnLo
PTO1_nOnHi
PTO1_nOffLo
PTO1_nOffHi
PTO1_Cl_Time
PTO1_vOnLo
PTO1_vOnHi
PTO1_vOffLo
PTO1_vOffHi
PowerTakeOffSlot2
PTO2_Var
PTO2_Switch
PTO2_Feedback
PTO2_Lamp
CAN16_BB_LampConfig
PTO2_Act
PTO2_Neut
PTO2_N_Time
PTO2_Park
PTO2_nOnLo
PTO2_nOnHi
PTO2_nOffLo
PTO2_nOffHi
PTO2_Cl_Time
PTO2_vOnLo
PTO2_vOnHi
PTO2_vOffLo
PTO2_vOffHi
PowerTake

In [18]:
X_sparse

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 33651 stored elements and shape (164, 11807)>

In [23]:
def get_dense_vector(document):
    """
    Convert sparse vector to dense format.
    """
    response = oclient.embeddings(model="nomic-embed-text", prompt=document)
    embeddings = response["embedding"]

    return embeddings


# 3. Upsert documents
for idx in range(X_sparse.shape[0]):
    row = X_sparse[idx]
    obj = function_group_objects[idx]
    document = documents[idx]
    indices = row.indices
    values = row.data

    client.upsert(
        collection_name=COLLECTION_NAME,
        points=[
            models.PointStruct(
                id=idx,
                payload={"json": obj.model_dump_json()},
                vector={
                    "text-sparse": models.SparseVector(
                        indices=indices.tolist(),
                        values=values.tolist(),
                    ),
                    "text-dense": get_dense_vector(document),
                },
            )
        ],
    )

## Query

In [24]:
# 4. Query preparation
query = "G. N.A.V"
query_vec = vectorizer.transform([query])
query_indices = query_vec.indices
query_values = query_vec.data

In [29]:
results = client.query_points(
    collection_name=COLLECTION_NAME,
    query=models.SparseVector(
        indices=query_indices,
        values=query_values
    ),
    using="text-sparse",
    limit=3,
)

In [31]:
print(results)

QueryResponse(
    points=[
        ScoredPoint(
            id=127,
            version=127,
            score=0.085372806,
            payload={
                'json': '{"name":"NavigationSupport","namePresentation":{"edt":"nfTxt","value":"Navigation support 
(navigeringsstöd)"},"description":null,"productVariantConditionRef":null,"ufNumber":[476],"propertyGroup":null,"pro
perty":[{"server":{"canAddress":"4C","propertyName":"GPSGLONASS"}}]}'
            },
            vector=None,
            shard_key=None,
            order_value=None
        ),
        ScoredPoint(
            id=126,
            version=126,
            score=0.084821515,
            payload={
                'json': 
'{"name":"DirectionIndicatorActivation","namePresentation":{"edt":"nfTxt","value":"Direction indicator 
activation"},"description":null,"productVariantConditionRef":{"ref":"ProductVariantCondition","value":"With_CUV1"},
"ufNumber":[51,53],"propertyGroup":null,"property":[{"server":{"canAddress":"1E","propertyName":"No_ofBulbsTurnFron
t"}},{"server":{"canAddress":"1E","propertyName":"NoTrailerDetection"}},{"server":{"canAddress":"17","propertyName"
:"DirectionIndicatorSound"}}]}'
            },
            vector=None,
            shard_key=None,
            order_value=None
        ),
        ScoredPoint(
            id=124,
            version=124,
            score=0.06593135,
            payload={
                'json': 
'{"name":"GasQuality","namePresentation":{"edt":"nfTxt","value":"GasQuality"},"description":null,"productVariantCon
ditionRef":null,"ufNumber":[0],"propertyGroup":null,"property":[{"server":{"canAddress":"01","propertyName":"GasQua
lityCompensation"}}]}'
            },
            vector=None,
            shard_key=None,
            order_value=None
        )
    ]
)